# VacationPy

In [4]:
pip install cartopy geoviews pyproj

     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     -------------- ------------------------- 20.5/56.0 kB ? eta -:--:--
     -------------------------------------- 56.0/56.0 kB 491.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.9 MB 2.6 MB/s eta 0:00:05
    --------------------------------------- 0.2/10.9 MB 2.8 MB/s eta 0:00:04
   - -------------------------------------- 0.5/10.9 MB 4.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/10.9 MB 4.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/10.9 MB 3.2 MB/s eta 0:00:04
   --- ------------------------------------ 0.8/10.9 MB 3.0 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/10.9 MB 2.9 MB/s eta 0:00:04
   ---- ----------------------------------- 1.2/10.9 MB 3.3 MB/s eta 0:00:03
   ---- ---------

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [37]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output/city_details.csv")

# Display sample data

city_data_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,puerto santa,-8.9906,-78.6456,64.02,80,91,9.08,PE,1723302598
1,1,bubaque,11.2833,-15.8333,78.84,87,100,20.24,GW,1723302598
2,2,port-aux-francais,-49.3500,70.2167,35.65,70,91,13.78,TF,1723302598
3,3,atafu village,-8.5421,-172.5159,82.20,78,61,17.54,TK,1723302599
4,4,balkanabat,39.5108,54.3671,100.89,23,0,4.63,TM,1723302599


---

### Step 1: Creating a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [38]:


humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrowing down the city_data_df DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values

#Selecting a temperature that lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 100) & (city_data_df["Max Temp"] > 30)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Windspeed"] < 60)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] < 100)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,puerto santa,-8.9906,-78.6456,64.02,80,91,9.08,PE,1723302598
2,2,port-aux-francais,-49.3500,70.2167,35.65,70,91,13.78,TF,1723302598
3,3,atafu village,-8.5421,-172.5159,82.20,78,61,17.54,TK,1723302599
6,6,blackmans bay,-43.0167,147.3167,57.72,61,32,5.01,AU,1723302599
7,7,sao joaquim,-28.2939,-49.9317,47.73,59,52,3.58,BR,1723302599
...,...,...,...,...,...,...,...,...,...,...
550,550,kapa'a,22.0752,-159.3190,77.32,85,40,13.80,US,1723302658
554,554,pisco,-13.7000,-76.2167,65.01,72,20,4.61,PE,1723302522
556,556,san miguel,13.4833,-88.1833,84.36,73,96,1.28,SV,1723302659
557,557,iquique,-20.2208,-70.1431,64.24,62,51,7.40,CL,1723302659


### Step 3: Create a new DataFrame called hotel_df.

In [42]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = pd.DataFrame({
    "City": city,
    "Country": country,
    "Lat": lat,
    "Lng": lng,
    "Humidity": humidity
})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto santa,PE,-8.9906,-78.6456,80,
2,port-aux-francais,TF,-49.3500,70.2167,70,
3,atafu village,TK,-8.5421,-172.5159,78,
6,blackmans bay,AU,-43.0167,147.3167,61,
7,sao joaquim,BR,-28.2939,-49.9317,59,
...,...,...,...,...,...,...
550,kapa'a,US,22.0752,-159.3190,85,
554,pisco,PE,-13.7000,-76.2167,72,
556,san miguel,SV,13.4833,-88.1833,73,
557,iquique,CL,-20.2208,-70.1431,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 100
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto santa - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
sao joaquim - nearest hotel: São Joaquim Park Hotel
cadale - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
luau - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
najran - nearest hotel: No hotel found
saurimo - nearest hotel: No hotel found
papao - nearest hotel: No hotel found
norfolk - nearest hotel: No hotel found
kayangel - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
gazli - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
luganville - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
qusmuryn - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
tinajo - nearest hotel: No hotel found
adamstown - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto santa,PE,-8.9906,-78.6456,80,No hotel found
2,port-aux-francais,TF,-49.3500,70.2167,70,No hotel found
3,atafu village,TK,-8.5421,-172.5159,78,No hotel found
6,blackmans bay,AU,-43.0167,147.3167,61,No hotel found
7,sao joaquim,BR,-28.2939,-49.9317,59,São Joaquim Park Hotel
...,...,...,...,...,...,...
550,kapa'a,US,22.0752,-159.3190,85,No hotel found
554,pisco,PE,-13.7000,-76.2167,72,No hotel found
556,san miguel,SV,13.4833,-88.1833,73,No hotel found
557,iquique,CL,-20.2208,-70.1431,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
# Configure the map plot
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)